### Imports

In [19]:
import os
import numpy as np
import pandas as pd

### Inputs

In [20]:
radnet_input_df = pd.read_csv("../inputs/radnet_input.csv")
radnet_input_apmr_edge = radnet_input_df['apmr_edge'].values
radnet_input_mfsr_edge = radnet_input_df['mfsr_edge'].values
radnet_input_idg_global = radnet_input_df['idg_global'].values

- apmr_edge (Active Prefix Message Ratio): Taxa de eficiência dos Active Prefixes no encaminhamento de mensagens através das arestas, 
medindo quão bem os campos de prefixo conseguem realizar matching probabilístico para encaminhar mensagens entre nós vizinhos.

- mfsr_edge (Message Forwarding Success Ratio): Proporção de mensagens transmitidas com sucesso através de uma aresta específica, 
considerando interferência, congestionamento e qualidade da conexão sem fio entre dois nós conectados.

- idg_global (Interest Group Delivery): Eficácia global da entrega de mensagens no modo Interest-Group addressing, 
medindo quão bem o protocolo consegue localizar e entregar mensagens aos nós que compartilham o mesmo interesse da aplicação.

In [21]:
inference_output_df = pd.read_csv("../outputs/inference_output.csv")
inference_output_class = inference_output_df['class'].values
inference_output_confidence = inference_output_df['confidence'].values

- confidence (Confiança da Inferência do Modelo GNN): Eficácia do modelo gerado pelo GNN medido na atapa anterior.

In [22]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Fusion

In [23]:
w_conf, w_apmr, w_mfsr, w_idg = 0.5, 0.2, 0.2, 0.1
w_sum = w_conf + w_apmr + w_mfsr + w_idg

In [24]:
fused = (
    w_conf*inference_output_confidence +
    w_apmr*radnet_input_apmr_edge +
    w_mfsr*radnet_input_mfsr_edge +
    w_idg*radnet_input_idg_global
) / w_sum

### Apply

In [25]:
cls  = np.where(fused < 0.5, 1 - inference_output_class, inference_output_class)
conf = np.where(fused < 0.5, 1 - fused, fused)

In [26]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[int(cls[i])]:<8} {conf[i]:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-83    8.3   100    0.72   0        Ruim     0.80      
-89    5.0   58     0.84   0        Ruim     0.80      
-69    5.7   33     0.57   0        Ruim     0.90      
-85    7.5   62     0.81   0        Ruim     0.84      
-88    4.0   36     0.81   0        Ruim     0.92      
-70    4.5   67     0.89   0        Ruim     0.94      
-84    9.3   42     0.73   0        Ruim     0.94      
-70    9.8   98     0.63   0        Ruim     0.78      
-65    9.8   42     0.43   0        Ruim     0.92      
-64    5.4   98     0.90   0        Ruim     0.93      
-74    7.2   40     0.91   0        Ruim     0.94      
-73    7.2   40     0.37   0        Ruim     0.91      
-71    9.0   74     0.58   0        Ruim     0.94      
-82    8.4   90     0.96   0        Ruim     0.82      
-61    6.2   49     0.44   0        Ruim     0.92      
-87    8.3   83     0.70   0        Ruim     0.9

### Save

In [27]:
fusion_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
fusion_output.to_csv("../outputs/fusion_output.csv", index=False)

print("Fusão de Dados salva em ../outputs/fusion_output.csv")

Fusão de Dados salva em ../outputs/fusion_output.csv
